# Text Summarization Pattern

In [1]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## Short Text With boto3

First we will summarize a short text, lets create a prompt

In [2]:
prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

Create the body for query

In [3]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

Invoking the model

In [4]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Here is a summary of the key points from the text:

- AWS announced Amazon Bedrock, a new service that provides API access to large language models (LLMs) from AI21 Labs, Anthropic, Stability AI, and Amazon. 

- Bedrock aims to make generative AI more accessible to developers by allowing them to easily integrate and deploy powerful text and image generation models into their applications.

- Bedrock offers serverless access to models including two new Titan LLMs from Amazon. 

- Key benefits of Bedrock include scalability, reliability, security, easy integration with AWS services, and the ability to customize models with your own data without managing infrastructure.


Streaming the output

In [5]:
modelId = 'anthropic.claude-v2'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"completion":" Here","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" is","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" a","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" summary","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" of","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" the","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" key","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" points","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" from","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" the","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" text","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":":","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":

Displaying the output in a more cosumable manner

In [6]:
from IPython.display import display_markdown,Markdown,clear_output

In [7]:
modelId = 'anthropic.claude-v2'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

 Here is a summary of the key points from the text:

- AWS announced Amazon Bedrock, a new service that provides API access to large language models (LLMs) from AI21 Labs, Anthropic, Stability AI, and Amazon. 

- Bedrock aims to make generative AI more accessible to developers by allowing them to easily integrate and deploy powerful text and image generation models into their applications.

- Bedrock offers serverless access to models including two new Titan LLMs from Amazon. 

- Key benefits of Bedrock include scalability, reliability, security, easy integration with AWS services, and the ability to customize models with your own data without managing infrastructure.

Same example using Claude3 Sonnet Model

In [8]:
messages=[{ "role":'user', "content":[{'type':'text','text': prompt}]}]

body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 512,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 1
        }  
    )  
    

In [9]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
#print(response_body)
output_list = response_body.get("content", [])
for output in output_list:
    print(output["text"])

Here is a summary of the provided text:

AWS has announced a new service called Amazon Bedrock that provides access to foundation models (FMs) from AI companies like AI21 Labs, Anthropic, Stability AI, and Amazon itself via an API. Bedrock aims to democratize access to powerful generative AI models for text and images, including Amazon's own new Titan large language models announced today.

Bedrock offers a serverless experience, allowing customers to easily find and use the right FM for their needs, customize models with their own data, and integrate them into applications using familiar AWS tools and services. This eliminates the need to manage infrastructure.

Key features of Bedrock include the ability to access a range of FMs, privately customize models, easily integrate them into applications, and leverage AWS capabilities like SageMaker Experiments to test models and SageMaker Pipelines to manage models at scale. Overall, Bedrock is designed to simplify building and scaling gene

In [10]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0", accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"type":"message_start","message":{"id":"msg_01QJrGJyZTZBgN2aFrDGkKQ1","type":"message","role":"assistant","content":[],"model":"claude-3-sonnet-28k-20240229","stop_reason":null,"stop_sequence":null,"usage":{"input_tokens":263,"output_tokens":1}}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_start","index":0,"content_block":{"type":"text","text":""}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"Here"}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" is"}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" a"}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" summary"}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" of"}}'}},
 {'chunk': {'bytes': b'{"type":"content_block_delta","i

In [11]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0", accept=accept, contentType=contentType)
for event in response.get("body"):
    chunk = json.loads(event["chunk"]["bytes"])

    if chunk['type'] == 'message_delta':
        print(f"\nStop reason: {chunk['delta']['stop_reason']}")
        print(f"Stop sequence: {chunk['delta']['stop_sequence']}")
        print(f"Output tokens: {chunk['usage']['output_tokens']}")

    if chunk['type'] == 'content_block_delta':
        if chunk['delta']['type'] == 'text_delta':
            print(chunk['delta']['text'], end="")

Here is a summary of the provided text:

AWS has announced a new service called Amazon Bedrock that provides access to foundation models (FMs) from AI companies like AI21 Labs, Anthropic, Stability AI, and Amazon itself via an API. Bedrock aims to make it easier for customers to build and scale generative AI applications using these powerful language models.

Key points:

- Bedrock offers access to a range of FMs for text and images through a scalable, secure AWS managed service.
- It includes Amazon's new Titan FMs, which are two new large language models announced today.
- Customers can easily find the right model, get started quickly, privately customize models with their own data, and integrate them into applications using familiar AWS tools.
- Bedrock provides a serverless experience without having to manage infrastructure.
- It integrates with Amazon SageMaker features like Experiments and Pipelines to test different models and manage them at scale.
- The goal is to democratize a

## Summarizing a Long Text

Specifying an LLM for LangChain Bedrock class

In [12]:
from langchain_aws import BedrockLLM
modelId = "amazon.titan-tg1-large"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=bedrock_client
)

### Loading text File with many Tokens

We will load the text file located in ./letters/2022-letter.txt and count the number of tokens in the file.
The text, too long to fit in the prompt, will be spited into smaller chunks with RecursiveCharacterTextSplitter.


In [22]:
#%pip install transformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

Now we have 10 documents and the first one has 439 tokens


### Summarizing the chunks
To summarize the chunks we can use three ways: 

- `stuff` puts all the chunks into one prompt. Thus, this would hit the maximum limit of tokens.
- `map_reduce`(calls the LLM multiple times) summarizes eacth chunk, combines the summary, and summarizes the combined summary. If the combined summary is too large, it would raise error. 
- `refine` (calls the LLM multiple times) summarizes chunks in sequence consolidating. 

Let´s try `map_reduce`


In [26]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error
    


In [27]:
print(output)

{'input_documents': [Document(page_content='As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.\n\nWhile there were an unusual number of simultaneous challenges this past year, the reality is that if you operate in large, dynamic, global market segments with many capable and well-funded competitors (the